In [28]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")  # This is to ignore any warnings that might pop up during execution

In [29]:
# Basic libraries to manipulate data
import pandas as pd                # data manipulation and wrangling
import numpy as np                 # numerical and scientific computing
import matplotlib.pyplot as plt    # general data visualization
import seaborn as sns              # interface for statistical visualization
import plotly.express as px
import sklearn                     # for machine learning
import statsmodels.api as sm       # for statistic models
import sqlite3                     # SQL data management
import os                          # for reading working directory
import matplotlib.ticker as mtick  # Import the ticker module for formatting

from scipy import stats	
from scipy.stats import f_oneway
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [30]:
np.random.seed(42)  # To ensure all the probabilistic things are reproducible

In [31]:
# Specify the path to the datasets
data_path = "./data/"  # Adjust this path to where your CSV files are stored

# Specify the filenames of the datasets
inflation_rate_filename = "inflationsheet.csv"
taxes_filename = "taxsheet.csv"
year_age_filename = "agesheet.csv"

# Read the CSV files and create DataFrames
inflation_rate_df = pd.read_csv(data_path + inflation_rate_filename)
taxes_df = pd.read_csv(data_path + taxes_filename)
year_age_df = pd.read_csv(data_path + year_age_filename)

In [32]:
def calculate_tax(monthly_interest, filtered_taxes_df):
    """Calculate the total tax based on the monthly interest and state-specific tax brackets."""
    state_tax = 0
    applicable_bracket = filtered_taxes_df[(filtered_taxes_df['Over'] <= monthly_interest) &
                                           ((filtered_taxes_df['Up to'] >= monthly_interest) | (pd.isna(filtered_taxes_df['Up to'])))]
    if not applicable_bracket.empty:
        bracket = applicable_bracket.iloc[-1]
        excess_rate = float(bracket['Excess percentage'].strip('%')) / 100
        excess_amount = monthly_interest - bracket['Over']
        state_tax = bracket['Base Tax'] + excess_amount * excess_rate

    # Calculate personal tax (5% of the monthly interest)
    personal_tax = monthly_interest * 0.05

    # Total tax
    total_tax = state_tax + personal_tax

    return total_tax

In [33]:
# Function definition remains the same...

def get_user_input(prompt, type_func=float):
    """Helper function to get and convert user input."""
    while True:
        try:
            return type_func(input(prompt))
        except ValueError:
            print("Invalid input, please try again.")

In [ ]:
def retirement_plan_calculator(initial_savings, savings_rate, initial_IRA, IRA_rate, initial_expense, age, start_year,
                               inflation_rate_df, filtered_taxes_df, year_age_df):
    savings = initial_savings
    IRA = initial_IRA
    expense = initial_expense
    year_savings_depleted = None

    yearly_data = []

    for year in range(start_year, 2051):
        age += 1

        # Adjusting for inflation
        if year != start_year:
            inflation_rate = inflation_rate_df[inflation_rate_df['Year'] == year]['Percentage'].values[0]
            expense *= (1 + inflation_rate)

        savings_interest = savings * savings_rate
        IRA_growth = IRA * IRA_rate
        IRA += IRA_growth

        monthly_savings_earned = savings_interest / 12

        total_tax = calculate_tax(monthly_savings_earned, filtered_taxes_df)

        savings = savings + savings_interest - total_tax - expense

        yearly_data.append({
            'Year': year,
            'Age': age,
            'Savings Balance': savings,
            'IRA Balance': IRA,
            'Annual Expense': expense,
            'Tax on Savings': total_tax,
            'End of Year Balance': savings
        })

        if savings < 0 and year_savings_depleted is None:
            year_savings_depleted = year

    yearly_details = pd.DataFrame(yearly_data)
    return yearly_details, year_savings_depleted

def calculate_percentage_change(original_value, new_value):
    """Calculate the percentage change."""
    return ((new_value - original_value) / original_value) * 100

def main():
    initial_savings = get_user_input("Enter initial savings amount (in thousands of $): ")
    savings_rate = get_user_input("Enter annual savings interest rate (as a decimal): ")
    initial_IRA = get_user_input("Enter initial IRA balance (in thousands of $): ")
    IRA_rate = get_user_input("Enter annual IRA growth rate (as a decimal): ")
    initial_expense = get_user_input("Enter initial annual expense (in thousands of $): ")
    age = get_user_input("Enter starting age: ", int)
    start_year = get_user_input("Enter starting year: ", int)
    state = input("Enter your state (e.g., MA for Massachusetts, CA for California): ").strip().upper()

    # Load tax data and filter by state
    filtered_taxes_df = taxes_df[taxes_df['State'] == state]

    # Calculate initial retirement plan
    initial_plan, year_savings_depleted = retirement_plan_calculator(
        initial_savings, savings_rate, initial_IRA, IRA_rate, initial_expense, age, start_year,
        inflation_rate_df, filtered_taxes_df, year_age_df)

    print("Initial Retirement Plan:")
    print(initial_plan)
    if year_savings_depleted:
        print(f"Savings will become negative in the year: {year_savings_depleted}")
    else:
        print("Savings did not become negative by the year 2050.")

    # Ask if the user wants to change expenses
    change_expense = input("Do you want to change expenses? (Y/N): ").strip().upper()
    if change_expense == "Y":
        new_annual_expense = get_user_input("Enter new reduced annual expense (in thousands of $): ")
        percentage_change = calculate_percentage_change(initial_expense, new_annual_expense)
        print(f"Annual expense changed by {percentage_change:.2f}%")

        # Recalculate with the new expense
        revised_plan, new_year_savings_depleted = retirement_plan_calculator(
            initial_savings, savings_rate, initial_IRA, IRA_rate, new_annual_expense, age, start_year,
            inflation_rate_df, filtered_taxes_df, year_age_df)

        print("Revised Retirement Plan with Reduced Expense:")
        print(revised_plan)
        if new_year_savings_depleted:
            print(f"Savings will become negative in the year: {new_year_savings_depleted}")
        else:
            print("Savings will not become negative by the year 2050 with the new plan.")

if __name__ == "__main__":
    main()

In [ ]:
def retirement_plan_calculator(initial_savings, savings_rate, initial_IRA, IRA_rate, initial_expense, age, start_year,
                               inflation_rate_df, filtered_taxes_df, year_age_df):
    savings = initial_savings
    IRA = initial_IRA
    expense = initial_expense
    year_savings_depleted = None

    yearly_data = []

    for year in range(start_year, 2051):
        age += 1

        # Adjusting for inflation
        if year != start_year:
            inflation_rate = inflation_rate_df[inflation_rate_df['Year'] == year]['Percentage'].values[0]
            expense *= (1 + inflation_rate)

        savings_interest = savings * savings_rate
        IRA_growth = IRA * IRA_rate
        IRA += IRA_growth

        monthly_savings_earned = savings_interest / 12

        total_tax = calculate_tax(monthly_savings_earned, filtered_taxes_df)

        savings = savings + savings_interest - total_tax - expense

        yearly_data.append({
            'Year': year,
            'Age': age,
            'Savings Balance': savings,
            'IRA Balance': IRA,
            'Annual Expense': expense,
            'Tax on Savings': total_tax,
            'End of Year Balance': savings
        })

        if savings < 0 and year_savings_depleted is None:
            year_savings_depleted = year

    yearly_details = pd.DataFrame(yearly_data)
    return yearly_details, year_savings_depleted

def calculate_percentage_change(original_value, new_value):
    """Calculate the percentage change."""
    return ((new_value - original_value) / original_value) * 100

def main():
    initial_savings = get_user_input("Enter initial savings amount (in thousands of $): ")
    savings_rate = get_user_input("Enter annual savings interest rate (as a decimal): ")
    initial_IRA = get_user_input("Enter initial IRA balance (in thousands of $): ")
    IRA_rate = get_user_input("Enter annual IRA growth rate (as a decimal): ")
    initial_expense = get_user_input("Enter initial annual expense (in thousands of $): ")
    age = get_user_input("Enter starting age: ", int)
    start_year = get_user_input("Enter starting year: ", int)
    state = input("Enter your state (e.g., MA for Massachusetts, CA for California): ").strip().upper()

    # Load tax data and filter by state
    filtered_taxes_df = taxes_df[taxes_df['State'] == state]

    # Calculate initial retirement plan
    initial_plan, year_savings_depleted = retirement_plan_calculator(
        initial_savings, savings_rate, initial_IRA, IRA_rate, initial_expense, age, start_year,
        inflation_rate_df, filtered_taxes_df, year_age_df)

    print("Initial Retirement Plan:")
    print(initial_plan)
    if year_savings_depleted:
        print(f"Savings will become negative in the year: {year_savings_depleted}")
    else:
        print("Savings did not become negative by the year 2050.")

    # Ask if the user wants to change expenses
    change_expense = input("Do you want to change expenses? (Y/N): ").strip().upper()
    if change_expense == "Y":
        new_annual_expense = get_user_input("Enter new reduced annual expense (in thousands of $): ")
        percentage_change = calculate_percentage_change(initial_expense, new_annual_expense)
        print(f"Annual expense changed by {percentage_change:.2f}%")

        # Recalculate with the new expense
        revised_plan, new_year_savings_depleted = retirement_plan_calculator(
            initial_savings, savings_rate, initial_IRA, IRA_rate, new_annual_expense, age, start_year,
            inflation_rate_df, filtered_taxes_df, year_age_df)

        print("Revised Retirement Plan with Reduced Expense:")
        print(revised_plan)
        if new_year_savings_depleted:
            print(f"Savings will become negative in the year: {new_year_savings_depleted}")
        else:
            print("Savings will not become negative by the year 2050 with the new plan.")

if __name__ == "__main__":
    main()

In [ ]:
def calculate_IRA_depletion(initial_IRA, annual_withdrawal, IRA_rate, start_year, inflation_rate_df, filtered_taxes_df):
    year_count = 0
    IRA_balance = initial_IRA

    while IRA_balance > 0:
        current_year = start_year + year_count
        inflation_rate = inflation_rate_df[inflation_rate_df['Year'] == current_year]['Percentage'].values[0]            

        year_count += 1
        IRA_balance *= (1 + IRA_rate)  # Apply growth rate
        adjusted_withdrawal = annual_withdrawal * (1 + inflation_rate)**year_count  # Adjust withdrawal for inflation
        
        # Calculate the tax for the year's withdrawal (assuming monthly withdrawal for tax calculation)
        tax_on_withdrawal = calculate_tax(adjusted_withdrawal / 12, filtered_taxes_df) * 12

        # Withdraw for the year after accounting for tax
        IRA_balance -= (adjusted_withdrawal + tax_on_withdrawal)

        if IRA_balance < 0:
            break

    return year_count

def main():
    initial_IRA = float(input("Enter initial IRA balance (in thousands of $): "))
    annual_withdrawal = float(input("Enter annual IRA withdrawal amount (in thousands of $): "))
    IRA_rate = float(input("Enter annual IRA growth rate (as a decimal): "))
    start_year = int(input("Enter the starting year: "))
    state = input("Enter your state (e.g., MA for Massachusetts, CA for California): ").strip().upper()

    # Load tax data and filter by state
    filtered_taxes_df = taxes_df[taxes_df['State'] == state]

    depletion_year = calculate_IRA_depletion(initial_IRA, annual_withdrawal, IRA_rate, start_year, inflation_rate_df, filtered_taxes_df)

    print(f"The IRA funds will be depleted in {depletion_year} years.")

if __name__ == "__main__":
    main()
